#### Project CCF pySpark RDD

In [2]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H"),("A","H"),("G","H"),("I","H"),("L","H"),("M","H"),("Z","J")],2)
r.collect()

Out[103]: [('A', 'B'),
 ('B', 'C'),
 ('B', 'D'),
 ('D', 'E'),
 ('F', 'G'),
 ('G', 'H'),
 ('A', 'H'),
 ('G', 'H'),
 ('I', 'H'),
 ('L', 'H'),
 ('M', 'H'),
 ('Z', 'J')]

In [3]:
from itertools import islice
g=sc.textFile("FileStore/tables/webGoogleS.txt")
#remove header 4 lines
data= g.mapPartitionsWithIndex(lambda idx, it: islice(it, 4, None) if idx == 0 else it)
data.count()

Out[83]: 41306

In [4]:
p = data.map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1]))
p.take(5)

Out[88]: [('0', '11342'),
 ('0', '824020'),
 ('0', '867923'),
 ('0', '891835'),
 ('11342', '0')]

In [5]:
def CCF_Iterate_reduce(pair):
  key, values = pair[0],list(pair[1])
  #global accum
  min = key
  valueL = []
  for value in values:
    if value < min:
       min = value
    valueL.append(value)
  if min < key:
    yield((key, min))
    for value in valueL:
      if min != value:
        accum.add(1)
        yield((value, min))

In [6]:
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)

In [7]:

new_pair_flag = True
iteration = 0
accum = sc.accumulator(0)
dedupJob = r

In [8]:
#for secondary sort

from pyspark.rdd import portable_hash
import itertools

n = 2 # number to hash
def partitioner(n):
   # Partition by the first item in the key tuple
    def partitioner_(x):
        return portable_hash(x[0]) % n
    return partitioner_


def unpair2(entry):
    return entry[0][0], entry[0][1]
  
def sorted_group(lines):
    return itertools.groupby(lines, key=lambda x: x[0])
  
def CCF_Iterate_reduce_SS_shuffle_test(pair):
  key, values = pair
  minvaluepair = next(values)
  minvalue=minvaluepair[1]
  
  if minvalue < key:
    #return key, minvalue
    yield key, minvalue
    for value in values:
     # if minvalue != value[1]:
      accum.add(1)
      yield value[1], minvalue


In [9]:
while new_pair_flag:
    iteration += 1
    newPair = False
    accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.flatMap(lambda e: (e,e[::-1]))
    #print(mapJob.collect())
    
    #Secondary Sort
    rddSS= (mapJob
      .keyBy(lambda kv: (kv[0], kv[1][0]))  # Create temporary composite key
      .repartitionAndSortWithinPartitions(numPartitions=n, partitionFunc=partitioner(n), ascending=True))
  #.map(lambda x: x[1]))  # Drop key (note: there is no partitioner set anymore)
    #for Secondary sort
    unpairedRDD = rddSS.map(unpair2, preservesPartitioning=True)
    groupedRDD = unpairedRDD.mapPartitions(sorted_group, preservesPartitioning=True)
    reduceJob = groupedRDD.flatMap(CCF_Iterate_reduce_SS_shuffle_test)
    # CCF-iterate (REDUCE)
   # reduceJob = rddSS.groupByKey().flatMap(lambda pair: CCF_Iterate_reduce_SS_shuffle(pair))#.sortByKey()
   # reduceJob = rddSS.groupByKey().flatMap(lambda pair: CCF_Iterate_reduce(pair))#.sortByKey()
    
  # CCF-dedup 
    dedupJob = reduceJob.distinct()

    # Force the RDD evalusation
    tmp = dedupJob.count()
    #print(dedupJob.collect())
    # Prepare next iteration
    #mapJob = dedupJob
    new_pair_flag = bool(accum.value)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag) + str(accum.value))
    #for analyse
    results = list(dedupJob.map(lambda e: e[::-1]).groupByKey().map(lambda x : (x[0],tuple(x[1]))).collect())
for k in results:
    print("Component id: "+str(k[0])+"| Number of nodes= "+str(len(k[1])+1))

Iteration :1 --- newPair : True10
Iteration :2 --- newPair : True15
Iteration :3 --- newPair : True4
Iteration :4 --- newPair : False0
Component id: J| Number of nodes= 2
Component id: A| Number of nodes= 11

In [10]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H"),("A","H"),("G","H"),("I","H")],2)
mapJob = r.flatMap(lambda e: (e,e[::-1]))
    #print(mapJob.collect())
    
    #Secondary Sort
n=3
rddSS= (mapJob
  .keyBy(lambda kv: (kv[0], kv[1][0]))  # Create temporary composite key
  .repartitionAndSortWithinPartitions(numPartitions=n, partitionFunc=partitioner(n), ascending=True))
  #.map(lambda x: x[1]))  # Drop key (note: there is no partitioner set anymore)
rddSS.glom().collect()


Out[92]: [[(('B', 'A'), ('B', 'A')),
 (('B', 'C'), ('B', 'C')),
 (('B', 'D'), ('B', 'D')),
 (('C', 'B'), ('C', 'B')),
 (('D', 'B'), ('D', 'B')),
 (('D', 'E'), ('D', 'E'))],
 [(('A', 'B'), ('A', 'B')),
 (('A', 'H'), ('A', 'H')),
 (('G', 'F'), ('G', 'F')),
 (('G', 'H'), ('G', 'H')),
 (('G', 'H'), ('G', 'H')),
 (('H', 'A'), ('H', 'A')),
 (('H', 'G'), ('H', 'G')),
 (('H', 'G'), ('H', 'G')),
 (('H', 'I'), ('H', 'I'))],
 [(('E', 'D'), ('E', 'D')),
 (('F', 'G'), ('F', 'G')),
 (('I', 'H'), ('I', 'H'))]]

In [11]:
def unpair2(entry):
    return entry[0][0], entry[0][1]
  
def sorted_group(lines):
    return itertools.groupby(lines, key=lambda x: x[0])

In [12]:
unpairedRDD = rddSS.map(unpair2, preservesPartitioning=True)

unpairedRDD.glom().collect()

Out[96]: [[('B', 'A'), ('B', 'C'), ('B', 'D'), ('C', 'B'), ('D', 'B'), ('D', 'E')],
 [('A', 'B'),
 ('A', 'H'),
 ('G', 'F'),
 ('G', 'H'),
 ('G', 'H'),
 ('H', 'A'),
 ('H', 'G'),
 ('H', 'G'),
 ('H', 'I')],
 [('E', 'D'), ('F', 'G'), ('I', 'H')]]

In [13]:
groupedRDD = unpairedRDD.mapPartitions(sorted_group, preservesPartitioning=True)
groupedRDD.glom().collect()

Out[97]: [[('B', <tuple_iterator at 0x7f42b15837b8>),
 ('C', <tuple_iterator at 0x7f42b15b7940>),
 ('D', <tuple_iterator at 0x7f42b15b7ef0>)],
 [('A', <tuple_iterator at 0x7f42b15b7dd8>),
 ('G', <tuple_iterator at 0x7f42b135a470>),
 ('H', <tuple_iterator at 0x7f42b135a550>)],
 [('E', <tuple_iterator at 0x7f42b15b7b00>),
 ('F', <tuple_iterator at 0x7f42b135aba8>),
 ('I', <tuple_iterator at 0x7f42b135a2e8>)]]

In [14]:
    # CCF-iterate (REDUCE)
accum = sc.accumulator(0)
reduceJob = groupedRDD.flatMap(CCF_Iterate_reduce_SS_shuffle_test)
reduceJob.collect()

Out[98]: [('B', 'A'),
 ('C', 'A'),
 ('D', 'A'),
 ('C', 'B'),
 ('D', 'B'),
 ('E', 'B'),
 ('G', 'F'),
 ('H', 'F'),
 ('H', 'F'),
 ('H', 'A'),
 ('G', 'A'),
 ('G', 'A'),
 ('I', 'A'),
 ('E', 'D'),
 ('I', 'H')]